<a href="https://colab.research.google.com/github/Apoak/307take2/blob/main/Perceptron_Algorithm_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Lab 2.2: Perceptron Algorithm in PyTorch

In this lab you will again implement the perceptron algorithm, but this time using PyTorch.

In [ ]:
!pip install torch

In [ ]:
import numpy as np
import torch

PyTorch is very similar to NumPy in its basic functionality.  In PyTorch arrays are called tensors.

In [ ]:
a = torch.tensor(5)
a

In [ ]:
b = torch.tensor(6)
a+b

In [ ]:
c = torch.zeros(3,5).float()
c

*A note on broadcasting:* You may have noticed in the previous lab that NumPy is particular about the sizes of the arrays in operations; PyTorch is the same way.

For example, if `A` has shape `(10,5)` and `b` has shape `(10,)`, then we can't compute `A*b`.  It wants the *last* dimensions to match, not the first ones.  So you would need to do either `A.T*b`.

In [ ]:
A = np.random.normal(size=(10,5))
b = np.ones(10)

In [ ]:
try:
    A*b
except ValueError as e:
    print(e)

In [ ]:
A.T*b

An alternative is to introduce an extra dimension of size one to $b$.  However, note that this produces the transposed result from before.

In [ ]:
A*b[:,None]

In [ ]:
A*np.expand_dims(b,-1)

In general, carefully check the sizes of all arrays in your code!

In [ ]:
!pip install scikit-learn palmerpenguins mlxtend

In [ ]:
from palmerpenguins import load_penguins
from mlxtend.plotting import plot_decision_regions
from matplotlib import pyplot as plt

Here we loading and format the Palmer penguins dataset for binary classification.

In [ ]:
df = load_penguins()

# drop rows with missing values
df.dropna(inplace=True)

# tricky code to randomly shuffle the rows
df = df.sample(frac=1).reset_index(drop=True)

# select only two specices
df = df[(df['species']=='Adelie')|(df['species']=='Chinstrap')]

# get two features
X = df[['flipper_length_mm','bill_length_mm']].values

# convert speces labels to -1 and 1
y = df['species'].map({'Adelie':-1,'Chinstrap':1}).values

To make the learning algorithm work more smoothly, we we will subtract the mean of each feature.

Here `np.mean` calculates a mean, and `axis=0` tells NumPy to calculate the mean over the rows (calculate the mean of each column).

In [ ]:
X -= np.mean(X,axis=0)

Now we will convert our `X` and `y` arrays to torch Tensors.

In [ ]:
X = torch.tensor(X).float()
y = torch.tensor(y).float()

In [ ]:
X

### Exercises

Your task is to again complete this class for the perceptron, with two changes from last time:
- the implementation should use PyTorch tensors, not NumPy arrays;
- `train_step` now accepts the entire dataset as input and should calculate the average gradient over all examples, rather than updating the weights one data point at a time.

In [ ]:
class Perceptron:
    def __init__(self,lr=1e-3):
        # store the learning rate
        self.lr = lr

        # initialize the weights to small, normally-distributed values
        self.w = torch.normal(mean=0, std=0.01, size=(2,))

        # initialize the bias to zero
        self.b = torch.zeros(1)

    def train_step(self,X:torch.Tensor,y:torch.Tensor) -> None:
        """ Apply the first update rule shown in lecture.
            Arguments:
             x: data matrix of shape (N,3)
             y: labels of shape (N,)
        """
        # WRITE CODE HERE
        z = X @ self.w.T + self.b
        # print(X.shape)
        # print(z.shape)
        # print(y.shape)
        gradient_sum = 0
        for n in range(X.shape[0]):
          for m in range(X.shape[1]):
            gradient_sum += (z - y) * X.T
        ave_grad = torch.mean(gradient_sum)

        # print(ave_grad.shape)
        self.w.T - self.lr * ave_grad
        # print(self.w)

    def predict(self,X:torch.Tensor) -> torch.Tensor:
        """ Calculate model prediction for all data points.
            Arguments:
             X: data matrix of shape (N,3)
            Returns:
             Predicted labels (-1 or 1) of shape (N,)
        """
        # WRITE CODE HERE
        z = X @ self.w + self.b
        return torch.where(z>0,1,-1)

    def score(self,X:torch.Tensor,y:torch.Tensor) -> torch.Tensor:
        """ Calculate model accuracy
            Arguments:
             X: data matrix of shape (N,3)
             y: labels of shape (N,)
            Returns:
             Accuracy score
        """
        # WRITE CODE HERE
        pred = self.predict(X)
        return torch.mean((pred == y).float())

# lr = 1e-3
# model = Perceptron(lr)
# model.train_step(X,y)

Run the following code to train the model and print out the accuracy at each step.

In [ ]:
lr = 1e-3
epochs = 100
model = Perceptron(lr)
for i in range(epochs):
    model.train_step(X,y)
    print(f'step {i}: {model.score(X,y)}')

QUESTION: The score values are drastically different each time, this seems like very abnormal behavior.

Run the training multiple times.  Is the training the same each time, or does it vary?  Why?

Play with the learning rate and number of epochs to find the best setting.